In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv',
                       parse_dates=['date_time'])

test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv',
                       parse_dates=['date_time'])

train_df

In [ ]:
train_df['hour'] = train_df['date_time'].dt.hour
test_df['hour'] = test_df['date_time'].dt.hour

train_df['day'] = train_df['date_time'].dt.day
test_df['day'] = test_df['date_time'].dt.day

train_df['week'] = train_df['date_time'].dt.dayofweek
test_df['week'] = test_df['date_time'].dt.dayofweek

train_df['month'] = train_df['date_time'].dt.month
test_df['month'] = test_df['date_time'].dt.month

train_df

In [ ]:
y = train_df.loc[ :, ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
train_df
X = train_df.drop(['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis=1)
test = test_df.drop(['date_time'], axis=1)
test

In [ ]:
total_df = pd.concat([X, test])
total_df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(total_df))
scaled_X = scaled_features[:X.shape[0]]
scaled_test = scaled_features[X.shape[0]:]
input_shape = scaled_X.shape[1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(scaled_X, y)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import HyperModel

from warnings import simplefilter
simplefilter = True

In [ ]:
class ANNHypetModel(HyperModel):
    
    def __init__(self, input_shape):
        self.input_shape = input_shape
        
    def build(self, hp):
        model = keras.Sequential()
        
        model.add(layers.Dense(
            units=hp.Int('units', 64, 256, 32, default=64),
            activation=hp.Choice('dense_activation',
                                values = ['relu', 'elu', 'tanh'],
                                default='relu'),
            activity_regularizer = keras.regularizers.l2(0.001),
            input_shape=[input_shape]))
        
        model.add(layers.Dropout(
            hp.Float('dropout',
                        min_value=0.0,
                        max_value=0.3,
                        default=0.05,
                        step=0.01)))
        
        model.add(layers.Dense(
            units=hp.Int('units', 128, 512, 64, default=128),
            activation=hp.Choice('dense_activation',
                                values = ['relu', 'elu', 'tanh'],
                                default='relu')))
        
        model.add(layers.Dropout(
            hp.Float('dropout',
                        min_value=0.0,
                        max_value=0.3,
                        default=0.05,
                        step=0.01)))
            
        model.add(layers.Dense(3, activation='relu'))
        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',
                            values=[0.01, 0.001, 0.0001]),
                            beta_1=0.9,
                            beta_2=0.999,
                            epsilon=1e-07),
                    loss='mae',
                    metrics=['accuracy'])
        
        
        return model
    
hypermodel = ANNHypetModel(input_shape)

In [ ]:
from keras_tuner.tuners import RandomSearch

tuner_rs = RandomSearch(hypermodel,
                           objective='val_accuracy',
                           max_trials=64
                       )

tuner_rs.search_space_summary()

In [ ]:
tuner_rs.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), verbose=2)

In [ ]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mae = best_model.evaluate(scaled_X, y)

In [ ]:
best_model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True
)

history = best_model.fit(
    scaled_X, y,
    batch_size=512,
    epochs=100,
    callbacks=[early_stop],
    verbose = 0
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'accuracy']].plot()

In [ ]:
y_pred = best_model.predict(scaled_test)

In [ ]:
submission = pd.DataFrame(y_pred, columns=['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], index=test_df['date_time'])
submission.to_csv('submission.csv')